Задача поиск похожих по эмбеддингам

Для выполнения 3 задания можно взять данные отсюда https://github.com/IlyaGusev/gazeta
Или этот датасет https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

Также подходят вопросы ответы от майл.

что надо сделать 
1. объединить в одну выборку
2. на основе word2vec/fasttext/glove/слоя Embedding реализовать метод поиска ближайших твитов

на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу.

3. Проверить насколько хорошо работают подходы

In [39]:
# !wget  https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

In [40]:
# !pip install corus

In [41]:
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import notebook

import pandas as pd

import gensim.downloader as api


import string
from sklearn.model_selection import train_test_split
import re
from tqdm import tqdm
from tqdm import tqdm_notebook

import gensim.models
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [42]:
### загрузить lenta-ru-news для векторизации

from corus import load_lenta
path = 'lenta-ru-news.csv.gz'
records = load_lenta(path)

In [43]:
next(records)

LentaRecord(
    url='https://lenta.ru/news/2018/12/14/cancer/',
    title='Названы регионы России с\xa0самой высокой смертностью от\xa0рака',
    text='Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости. По словам Голиковой, чаще всего онкологические заболевания становились причиной смерти в Псковской, Тверской, Тульской и Орловской областях, а также в Севастополе. Вице-премьер напомнила, что главные факторы смертности в России — рак и болезни системы кровообращения. В начале года стало известно, что смертность от онкологических заболеваний среди россиян снизилась впервые за три года. По данным Росстата, в 2017 году от рака умерли 289 тысяч человек. Это на 3,5 процента меньше, чем годом ранее.',
    topic='Россия',
    tags='Общество',
    date=None
)

In [44]:
# Создадим корпус текстов

# возьмем заголовок текста
corpus_title = []
# возьмем сам текст
corpus_text = []

for num, iter in tqdm(enumerate(records)):
    corpus_title.append(iter.title)
    corpus_text.append(iter.text)
    if num == 99999:
        break

10254it [00:00, 31049.31it/s]


KeyboardInterrupt: 

In [ ]:
corpus_title[10], corpus_text[10]

('Овечкин повторил свой рекорд',
 ' Капитан «Вашингтона» Александр Овечкин сделал хет-трик в игре Национальной хоккейной лиги (НХЛ) с «Каролиной». Таким образом он продлил результативную серию до 13 матчей и повторил личный рекорд января-февраля 2007 года. Об этом сообщает Sports.ru. Встреча завершилась со счетом 5:6 Б — «Вашингтон» переиграл «Каролину» по булитам. Для россиянина это второй подряд хет-трик. До этого он отметился им в матче против «Детройт Ред Уингс». Клубный рекорд остается за Майклом Гартнером — 17 матчей. В соперничестве бомбардиров Овечкин занял пятое место с 42 (28+14) очками в 31 матче.')

In [ ]:
df = pd.DataFrame(corpus_title, columns=['title'])
df['text'] = corpus_text
df.head()

,title,text
0,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...
1,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...
2,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...
3,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...
4,Архиепископ канонической УПЦ отказался прийти ...,Архиепископ канонической Украинской православн...


In [ ]:
# Preprocess for models fitting

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [ ]:
# Функция преобразования (лемматизация, удаление стоп-слов и пунктуации)
def preprocess_txt(line):

    # Remove URLs
    line = re.sub(r'http\S+|www\S+|https\S+', '', line, flags=re.MULTILINE)
    # Remove punctuation
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    # Lemmatize and lowercase words
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    # Remove stop words  and empty strings
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [ ]:
# создание списка токенов

sentences = []
counter = 0

for line in tqdm_notebook(df.text):
    spls = preprocess_txt(line)
    sentences.append(spls)
    counter += 1      
    if counter > 100000:
        break

  0%|          | 0/100000 [00:00<?, ?it/s]

In [ ]:
sentences[0]

['австрийский',
 'правоохранительный',
 'орган',
 'представить',
 'доказательство',
 'нарушение',
 'российский',
 'биатлонист',
 'антидопинговый',
 'правило',
 'сообщить',
 'посол',
 'россия',
 'вена',
 'дмитрий',
 'любинский',
 'итог',
 'встреча',
 'уполномоченный',
 'адвокат',
 'дипмиссия',
 'представитель',
 'прокуратура',
 'страна',
 'передавать',
 'тасс',
 '«действовать',
 'презумпция',
 'невиновность',
 'какихлибо',
 'ограничение',
 'свобода',
 'передвижение',
 'команда',
 'нет»',
 '—',
 'добавить',
 'посольство',
 'международный',
 'союз',
 'биатлонист',
 'ibu',
 'применять',
 'санкция',
 'российский',
 'биатлонист',
 'продолжить',
 'выступление',
 'кубок',
 'мир',
 'полиция',
 'нагрянуть',
 'отель',
 'сборная',
 'россия',
 'хохфильцен',
 'вечером',
 '12',
 'декабрь',
 'написать',
 'биатлонист',
 'александр',
 'логинов',
 'считать',
 'виновный',
 'махинация',
 'переливание',
 'кровь',
 'биатлонист',
 'антон',
 'шипулина',
 'попасть',
 'список',
 'полиция',
 'нанести',
 'отдельны

In [ ]:
modelW2V = Word2Vec(sentences=sentences, vector_size=200, window=5, min_count=1)

In [ ]:
modelFT = FastText(sentences=sentences, vector_size=200, min_count=1, window=5, workers=8)

In [ ]:
# w2v_index = annoy.AnnoyIndex(300 ,'angular')
# ft_index = annoy.AnnoyIndex(300 ,'angular')

# index_map = {}
# counter = 0


# for line in tqdm_notebook(df.text):
#     n_w2v = 0
#     n_ft = 0
#     spls = line.split("\t")
#     index_map[counter] = spls[1]
#     question = preprocess_txt(spls[0])
    
#     vector_w2v = np.zeros(300)
#     vector_ft = np.zeros(300)
#     for word in question:
#         if word in modelW2V.wv:
#             vector_w2v += modelW2V.wv[word]
#             n_w2v += 1
#         if word in modelFT.wv:
#             vector_ft += modelFT.wv[word]
#             n_ft += 1
#     if n_w2v > 0:
#         vector_w2v = vector_w2v / n_w2v
#     if n_ft > 0:
#         vector_ft = vector_ft / n_ft
#     w2v_index.add_item(counter, vector_w2v)
#     ft_index.add_item(counter, vector_ft)
        
#     counter += 1
    
#     if counter > 100000:
#         break

# w2v_index.build(10)
# ft_index.build(10)

<ipython-input-16-64550af31b99>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(f):


True

первый способ - просто усреднение умбедингов по токенам
Второй способ - все токены перевзвесить через idf

In [47]:
w2v_index = annoy.AnnoyIndex(200, 'angular')
ft_index = annoy.AnnoyIndex(200, 'angular')

index_map = {}

for counter, line in tqdm_notebook(df.text.head(100000).items()):  # Use head to limit the number of items
    index_map[counter] = line
    question = preprocess_txt(line)

    # Calculate the average word2vec and fasttext vectors for the question
    vector_w2v = np.mean([modelW2V.wv[word] for word in question if word in modelW2V.wv], axis=0)
    vector_ft = np.mean([modelFT.wv[word] for word in question if word in modelFT.wv], axis=0)

    # In case any of the vectors are empty (all zeros), replace them with np.zeros(300)
    vector_w2v = np.zeros(200) if np.isnan(vector_w2v).any() else vector_w2v
    vector_ft = np.zeros(200) if np.isnan(vector_ft).any() else vector_ft

    w2v_index.add_item(counter, vector_w2v)
    ft_index.add_item(counter, vector_ft)

w2v_index.build(10)
ft_index.build(10)

0it [00:00, ?it/s]

True

In [48]:
# 5 ближайших

def get_response(question, index, model, index_map):
    question = preprocess_txt(question)
#     print(question)
    vector = np.zeros(200)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, )
    return [index_map[i] for i in answers]

In [49]:
TEXT = "чемпион мира по футболу"

In [50]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['Футболист французского ПСЖ и сборной Бразилии Неймар рассказал о готовности национальной команды к чемпионату мира 2018 года на Youtube-канале Desimpedidos. «Эта сборная Бразилии готова к турниру лучше, чем четыре года назад. Выиграть чемпионат мира — самая главная цель. Сейчас этот турнир намного важнее Лиги чемпионов», — заявил форвард. Футболист также определил свое место среди лучших игроков мира. «Есть двое, которые сейчас выше всех остальных, — это Криштиану Роналду и Лионель Месси. При всей своей скромности могу сказать, я — лучший игрок в мире на данный момент. Потому что они с другой планеты», — добавил он. На домашнем чемпионате мира в 2014 году сборная Бразилии дошла до полуфинала, где была разгромлена командой Германии со счетом 1:7. В матче за третье место против Нидерландов хозяева турнира также потерпели поражение (0:3). Чемпионат мира в России начался 14 июня. Сборная Бразилии на групповом этапе турнира сыграет с командами Швейцарии, Коста-Рики и Сербии.',
 'Защитник 

In [51]:
get_response(TEXT, ft_index, modelFT, index_map)

['Российский биатлонист Максим Цветков выиграл масс-старт на последнем этапе Кубка мира этого сезона в Тюмени, преодолев дистанцию без единого промаха. Об этом сообщает корреспондент «Ленты.ру». Второе место занял норвежский биатлонист Эрленн Бьентегорд. Тройку призеров замкнул еще один норвежец Йоханнес Бе. В гонке принимало участие шесть представителей сборной России, в том числе и лидер сборной Антон Шипулин. Масс-старт в Тюмени стал последней мужской биатлонной гонкой в рамках Кубка мира в этом сезоне. До победы Цветкова на пьедестал Кубка мира удавалось взобраться только Шипулину, который одержал победу в спринте на этапе Кубка мира в финском Контиолахти. Для 26-летнего Цветкова триумф стал первой победой в карьере на этапах биатлонного Кубка мира.',
 'Сборная России после поражения от команды Хорватии в четвертьфинале чемпионата мира обратилась к болельщикам в Twitter. «Друзья! Мы покидаем турнир со слезами на глазах, но с высоко поднятой головой. Спасибо вам за те искренние, неп

In [52]:
TEXT = "экологические движения"

In [53]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['Правительство России утвердило изменения в Правила дорожного движения, сообщается на сайте кабинета министров. Соответствующее постановление было подписано 12 июля. В частности, в связи с ускоренным развитием экологически чистого вида транспорта с целью обозначения мест зарядки электромобилей и обеспечения возможности их парковки в таких местах введены новые термины «электромобиль», «гибридный автомобиль», соответствующие дорожные знаки и разметки. В целях защиты окружающей среды от воздействия выбросов автомобилями вредных веществ введены новые дорожные знаки и термины. Кроме того, уточнены определения применительно к ситуации, когда необходимо произвести разметку проезжей части при наличии трамвайных путей и островков безопасности. В целях обеспечения безопасности дорожного движения действие дорожного знака «Остановка запрещена» распространено на маршрутные транспортные средства вне зоны остановок маршрутного транспорта. Также изменениями в Правила разрешается движение велосипедист

In [54]:
get_response(TEXT, ft_index, modelFT, index_map)

['Зампредседателя комитета Совета Федерации по аграрно-продовольственной политике и природопользованию Ирина Гехт  предложила закрепить в законодательстве понятие «экологическое страхование», передает Агентство городских новостей «Москва». «Я считаю, что в законодательстве должно быть внесено положение о том, что, когда проектируется тот или иной объект, крупный и промышленный в том числе, должна закладываться, так называемая часть экологического страхования. Для того чтобы после реализации проекта были деньги на рекультивацию территорий, на восстановление и компенсацию того вреда, который был нанесен окружающей среде», — сказала она в ходе круглого стола на тему: «Протестные экологические акции: кому выгодно?» Сенатор добавила, что подобная практика «применяется во всем мире». Гехт уточнила, что проект должен содержать, в том числе, и стоимостную оценку экологических рисков. На круглом столе был представлен экспертный доклад «Экозащита или эконападение: политические экологи в России и